# Object Following

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Set Network configuration

In [ ]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

In [ ]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
ROS_MASTER_IP=ip_address
# ROS_MASTER_IP="192.168.10.14" # custom

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)
print("ROS_MASTER_IP is {}".format(ROS_MASTER_IP))

### Check environment  

It should outputs like this:

```
192.168.10.23
http://192.168.10.14:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [ ]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

## Import libraries

In [ ]:
from sidecar import Sidecar
# Using jupyros(jupyter-ros) 0.5.0 from conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

In [ ]:
from tflite_runtime.interpreter import Interpreter
import numpy as np
import cv2
from matplotlib import pyplot as plt

import tflite_runtime
print(tflite_runtime.__version__)

## Load Tensor Flow Lite

In [ ]:
# Load model
interpreter = Interpreter(model_path="coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/detect.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5
# Check output layer name to determine if this model was created with TF2 or TF1,
# because outputs are ordered differently for TF2 and TF1 models
outname = output_details[0]['name']

if ('StatefulPartitionedCall' in outname): # This is a TF2 model
    boxes_idx, classes_idx, scores_idx = 1, 3, 0
else: # This is a TF1 model
    boxes_idx, classes_idx, scores_idx = 0, 1, 2

print(input_details, output_details, height, width)

In [ ]:
# Load the label map
with open("coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/labelmap.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

In [ ]:
# Check labels
print(labels)

## Subscribe camera image and control robot

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /image_raw`, re-run the network configuration step.

In [ ]:
!rostopic info /image_raw

### Prepare publisher

In [ ]:
import jupyros
import rospy
from sensor_msgs.msg import Image
from geometry_msgs.msg import Pose

rospy.init_node("image_subscriber")
pub = rospy.Publisher('/body_pose', Pose, queue_size=10)

### Create widget

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=88, description='tracked label')
yaw_widget = widgets.FloatSlider(min = -0.3, max = 0.3, step = 0.005, description='yaw_increment')
pitch_widget = widgets.FloatSlider(min = -0.2, max = 0.2, step = 0.005, description='pitch_increment')

try:
    sc = Sidecar(title='Object Following Output')
    with sc:
        display(widgets.VBox([
            widgets.HBox([image_widget, pitch_widget]),
            label_widget,
            yaw_widget
        ]))
except:
    display(widgets.VBox([
        widgets.HBox([image_widget, pitch_widget]),
        label_widget,
        yaw_widget
    ]))

In [ ]:
import cv2
from cv_bridge import CvBridge, CvBridgeError
from io import BytesIO
import datetime
import math

bridge = CvBridge()

rospy.init_node("image_subscriber")
rate = rospy.Rate(5)

min_conf_threshold = 0.5
move_threshold = 10

def callback(msg):
    if rate.remaining() > rospy.Duration(0):
        return
    try:
        image = bridge.imgmsg_to_cv2(msg, "bgr8")
    except CvBridgeError as e:
        rospy.logerr("CvBridge Error: {0}".format(e))
    # Load image and resize to expected shape [1xHxWx3]
    imH, imW, _ = image.shape 
    image_resized = cv2.resize(image, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if floating_model:
        input_data = (np.float32(input_data) - input_mean) / input_std

    # Perform the actual detection by running the model with the image as input
    interpreter.set_tensor(input_details[0]['index'],input_data)
    interpreter.invoke()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[boxes_idx]['index'])[0] # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[classes_idx]['index'])[0]+1 # Class index of detected objects
    scores = interpreter.get_tensor(output_details[scores_idx]['index'])[0] # Confidence of detected objects

    # Loop over for first 5 detections and draw detection box if confidence is above minimum threshold
    if sum(x>min_conf_threshold for x in scores) > 5:
        print("too many objects!")
        return
    direction = None
    for i in range(len(scores)):
        if ((scores[i] > min_conf_threshold) and (scores[i] <= 1.0)):

            # Get bounding box coordinates and draw box
            # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
            ymin = int(max(1,(boxes[i][0] * imH)))
            xmin = int(max(1,(boxes[i][1] * imW)))
            ymax = int(min(imH,(boxes[i][2] * imH)))
            xmax = int(min(imW,(boxes[i][3] * imW)))
            

            cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

            # Draw label
            object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
            label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 2.5, 4) # Get font size
            label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
            cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
            cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 0, 0), 4) # Draw label text
            
 
            if classes[i] == int(label_widget.value):
                xcenter = (xmax+xmin)/2
                ycenter = (ymax+ymin)/2
                print("Label, (X, Y), max(X, Y) = {}, ({}, {}), ({}, {})".format(object_name, xcenter, ycenter, imW, imH))
                yaw_widget.value = 0.0003 * (imW/2 - xcenter) # 640px == 0.2 rad
                pitch_widget.value = -0.0002 * (imH/2 - ycenter) # 480px == 0.1rad


    _, frame = cv2.imencode('.jpg', cv2.resize(image, (int(width), int(height))), [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    frame_buffer = BytesIO(frame)
    image_widget.value = frame_buffer.getvalue()
    last_time_stamp = datetime.datetime.now()

In [ ]:
%%thread_cell

rate = rospy.Rate(20)
while not rospy.is_shutdown():
    pose = Pose()

    yaw = yaw_widget.value
    pitch = pitch_widget.value
    roll = 0

    cy = math.cos(yaw)
    sy = math.sin(yaw)
    cp = math.cos(pitch)
    sp = math.sin(pitch)
    cr = math.cos(roll)
    sr = math.sin(roll)

    pose.orientation.w = cy * cp * cr + sy * sp * sr
    pose.orientation.x = cy * cp * sr - sy * sp * cr
    pose.orientation.y = sy * cp * sr + cy * sp * cr
    pose.orientation.z = sy * cp * cr - cy * sp * sr

    pub.publish(pose)
    rate.sleep()

In [ ]:
jupyros.subscribe("/image_raw", Image, callback)

In [ ]:
!rostopic info /body_pose